# Project Code

In [ ]:
import cv2
import numpy as np
open_kernel = np.ones((5,5),np.uint8)
close_kernel = np.ones((7,7),np.uint8)
dilation_kernel = np.ones((10, 10), np.uint8)
#cv2.MORPH_CLOSE simply removes unnecessary details that are not required like in this project the noise is small black holes appear on the screen 
#cv2.MORPH_OPEN will remove the white region on the boundary of the cloth that is not required
#cv2.dilate increases the white region in the image or the size of the foreground object increases
    
def filter_mask(mask):
    close_mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, close_kernel)
    open_mask = cv2.morphologyEx(close_mask, cv2.MORPH_OPEN, open_kernel)
    dilation = cv2.dilate(open_mask, dilation_kernel, iterations= 1)
    return dilation

def nothing(x):
    pass

cap=cv2.VideoCapture(0)
bars=cv2.namedWindow("bars")
#Trackbars in OpenCV are helpful to tweak a variable value instantly without closing and relaunching the program
cv2.createTrackbar("upper_hue","bars",135,180,nothing)
cv2.createTrackbar("upper_saturation","bars",255,255,nothing)
cv2.createTrackbar("upper_value","bars",255,255,nothing)
cv2.createTrackbar("lower_hue","bars",110,180,nothing)
cv2.createTrackbar("lower_saturation","bars",80,255,nothing)
cv2.createTrackbar("lower_value","bars",50,255,nothing)

# for taking initial frame
for i in range(50):
    ret, init_frame= cap.read()
cv2.imshow("initial frame",init_frame)
#while(True):
#    cv2.waitKey(1000)
#    ret, init_frame= cap.read()
#    if (ret):
#        cv2.imshow("initial frame",init_frame)
#        break


# start capturing frames for actual magic!!!!!!!!!!!        
while(True):
    ret, frame= cap.read()
    # we want to convert our BGR image into HSV because the method that we are using to extract a color requires HSV Space
    hsv_img=cv2.cvtColor(frame,cv2.COLOR_BGR2HSV)
    upper_hue=cv2.getTrackbarPos("upper_hue","bars")
    upper_saturation=cv2.getTrackbarPos("upper_saturation","bars")
    upper_value=cv2.getTrackbarPos("upper_value","bars")
    lower_hue=cv2.getTrackbarPos("lower_hue","bars")
    lower_saturation=cv2.getTrackbarPos("lower_saturation","bars")
    lower_value=cv2.getTrackbarPos("lower_value","bars")
    #kernel is the matrix that are used for dilation, 
    #it tells us how to change the value of any given pixel by combining it with different amounts of the neighbouring pixels
    #dilate means we are trying to reduce impurities
    #np.uint8 is an unsigned 8bit integer
    kernel= np.ones((3,3),np.uint8)
    lower_hsv = np.array([lower_hue,lower_saturation,lower_value])
    upper_hsv = np.array([upper_hue,upper_saturation,upper_value])
    mask=cv2.inRange(hsv_img,lower_hsv,upper_hsv)
    #mask=cv2.morphologyEx(mask, cv2.MORPH_OPEN ,np.ones((3,3) , np.uint8) , iterations=2)
    #mask=cv2.morphologyEx(mask, cv2.MORPH_DILATE ,np.ones((3,3) , np.uint8) , iterations=1)
    mask=filter_mask(mask)
    #mask tells us which area of the image should keep
    #masks are binary images that indicate the pixels in which operation is performed
    #mask=cv2.medianBlur(mask,5)
    mask_inv=cv2.bitwise_not(mask)
    cv2.imshow("COLOR",mask)
    
    b=frame[:,:,0]
    g=frame[:,:,1]
    r=frame[:,:,2]
    b=cv2.bitwise_and(mask_inv,b)
    g=cv2.bitwise_and(mask_inv,g)
    r=cv2.bitwise_and(mask_inv,r)
    #it will return a frame with black value of choosen color
    frame_inv= cv2.merge([b,g,r])
    
    b=init_frame[:,:,0]
    g=init_frame[:,:,1]
    r=init_frame[:,:,2]
    b=cv2.bitwise_and(b,mask)
    g=cv2.bitwise_and(g,mask)
    r=cv2.bitwise_and(r,mask)
    cloak_area= cv2.merge([b,g,r])
    
    final= cv2.bitwise_or(frame_inv,cloak_area)
    
    cv2.imshow("Magical cloak",final)
    cv2.imshow("original",frame)
    
    if(cv2.waitKey(3)==ord('q')):
        break
        
        
    

cv2.destroyAllWindows()
cap.release()
